In [1]:
import pandas as pd
from PIL import Image
import io
import ast
import cv2
import numpy as np
import random
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import corpus_bleu
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

import string
import cv2
import os
import glob
from PIL import Image
from time import time

from tensorflow.keras import Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Dense, Activation, Flatten, Reshape, Dropout
# from tensorflow.keras.layers.wrappers import Bidirectional
# from tensorflow.keras.layers.merge import add
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import re
from sklearn.model_selection import train_test_split
import pickle

In [2]:
def preprocess(datas):
    return_data = []
    for data in datas:
        data=data.lower()
        data=data.replace('\\r','').replace('\\n','').replace('\\',' ').replace("n\'t",' not').replace(',',' ')\
        .replace('.',' ').replace('%',' ').replace("'s",' is').replace('-',' ').replace('"','').replace('_','').replace(':','')\
        .replace(';','').replace('!','').replace('!!','').replace(';','').replace('/',' ').replace('?',' ')

        p = re.compile('[0-9]+')
        data=p.sub('', data)

        data = re.sub(r"won't", "will not", data)
        data = re.sub(r"can\'t", "can not", data)

        # general
        data = re.sub(r"n\'t", " not", data)
        data = re.sub(r"\'re", " are", data)
        data = re.sub(r"\'s", " is", data)
        data = re.sub(r"\'d", " would", data)
        data = re.sub(r"\'ll", " will", data)
        data = re.sub(r"\'t", " not", data)
        data = re.sub(r"\'ve", " have", data)
        data = re.sub(r"\'m", " am", data)

#         data = ' '.join(e for e in data.split() if e.lower() not in stopwords)
        # data=data.lower()
        data=' '.join(data.split())
        return_data.append(data)
        
    return return_data

In [3]:
data_df = pd.read_csv("final_df")
df_valid = pd.read_csv("final_valid")

In [4]:
data_df['captions']=preprocess(data_df['captions'].values)

In [5]:
data_df['enc_input'] = 'SOS '+data_df['captions']
data_df['enc_output'] = data_df['captions']+' EOS'

df_valid['enc_input'] = 'SOS '+df_valid['captions']
df_valid['enc_output'] = df_valid['captions']+' EOS'

In [6]:
X_train = data_df[['image','enc_input']]
y_train = data_df['enc_output']

X_test = df_valid[['image','enc_input']]
y_test = df_valid['enc_output']

In [7]:
y_train.values+' '+X_train['enc_input'].values

array(['many aircraft are parked next to a long building in an airport EOS SOS many aircraft are parked next to a long building in an airport',
       'many planes are parked next to a long building at an airport EOS SOS many planes are parked next to a long building at an airport',
       'many planes are parked next to a long building in an airport EOS SOS many planes are parked next to a long building in an airport',
       ...,
       'two baseball grounds are next to a road and a lake EOS SOS two baseball grounds are next to a road and a lake',
       'two baseball fields are next to a road and a pool EOS SOS two baseball fields are next to a road and a pool',
       'two baseball fields are next to a road and a lake EOS SOS two baseball fields are next to a road and a lake'],
      dtype=object)

In [46]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(y_train.values+' '+X_train['enc_input'].values)

In [8]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer1 = pickle.load(handle)

In [9]:
tokenizer1.word_index["sos"]

4

In [10]:
len_of_captions = [len(i.split()) for i in X_train['enc_input'].values]

In [11]:
embeddings_index = dict()
f = open('F:\Data Science\data\glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [12]:
eng_inp_vocab_size=max(tokenizer1.index_word.keys())
embedding_matrix = np.zeros((eng_inp_vocab_size+1, 100))
for word, i in tokenizer1.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
class Dataset:
    def __init__(self, data,model_im_ex, embedding, tokenizer_eng, max_len, vocab_size):
        self.decoder_inps = data[0]['enc_input'].values
        self.images = data[0]['image'].values
        self.model_im_ex = model_im_ex
#         self.decoder_inps = data['decoder_input_hin'].values
        self.decoder_outs = data[1].values
        self.embedding = embedding
#         self.embedding1 = embedding1
        self.tknizer_eng = tokenizer_eng
#         self.tknizer_hin_inp = tokenizer_hin_inp
#         self.tknizer_hin = tokenizer_hin
        self.max_len = max_len
        self.vocab_size=vocab_size

    def __getitem__(self, i):
        
        img = cv2.imread(self.images[i])
        img = cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)
        img = img/255.0
        
        img = self.model_im_ex(np.array([img]))[0]
        
        
        
        
        self.decoder_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]]) # need to pass list of values
#         self.decoder_inp_seq = self.tknizer_hin_inp.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])
#         print(self.decoder_inp_seq)
        self.decoder_seq = pad_sequences(self.decoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        
#         self.decoder_out_seq = pad_sequences([self.decoder_inp_seq[0][1:]], maxlen=25, dtype='int32', padding='post')
        #print(self.decoder_inp_seq1)
#         print(self.decoder_seq)
#         print(self.decoder_out_seq)
        #print(self.encoder_seq)
        self.decoder_seq1=[]
        for j in self.decoder_seq[0]:
            self.decoder_seq1.append(self.embedding[j])
            
            
#         self.decoder_out_seq1=[]
#         for i in self.decoder_out_seq[0]:
#             self.decoder_out_seq1.append(self.embedding[i])    
        #print(np.array(encoder_seq1).shape)
        
#         decoder_inp_seq1 =[]
        
#         for i in self.decoder_inp_seq[0]:
#             decoder_inp_seq1.append(self.embedding1[i])
            
#         self.decoder_out_seq1 = np.zeros((25,self.vocab_hindi+1),dtype=np.float32)
        
#         for i,d in enumerate(self.decoder_out_seq[0]):
#             self.decoder_out_seq1[i,d]=1
        
        
        
        #print(self.encoder_seq.shape)
#         print(img)
        return np.array(img), np.array(self.decoder_seq1), np.array(self.decoder_out_seq)

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        #print(dataset)
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.decoder_inps))
        #print(self.indexes)


    def __getitem__(self, i):
        #print(i,'in dataloader')
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            #print(self.dataset[j])
            data.append(self.dataset[j])
            #break
        #print(len(data),'in dataloader')
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
#         print('batch')
        #print(batch[2].shape)
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],tf.squeeze(batch[2])])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [14]:
vocab_size = len(tokenizer1.index_word)
vocab_size

2693

In [15]:
base_model = VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=layers.Input((224,224,3)),
)
base_model.trainable = False

input1 = base_model.input
output1 = base_model.layers[-1].output

image_feature_model = tf.keras.Model(input1,output1)

In [43]:
image_feature_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
train_dataset = Dataset((X_train,y_train),image_feature_model, embedding_matrix, tokenizer1, 20,vocab_size)
test_dataset  = Dataset((X_test,y_test),image_feature_model,embedding_matrix, tokenizer1,20,vocab_size)
BATCH_SIZE=16
train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = Dataloder(test_dataset, batch_size=BATCH_SIZE)

In [17]:
train_dataloader[0][0][0].shape

(16, 7, 7, 512)

In [18]:
len(train_dataloader[0][0][0][0])

7

In [19]:
train_dataloader[0][0][1].shape

(16, 20, 100)

In [20]:
len(train_dataloader[0][1][0].shape)

1

In [21]:
train_dataloader[0][1][0]

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([  6, 187,   2,  48,  49,  20,   1, 119,  37,  11,  26,  54,   3,
         0,   0,   0,   0,   0,   0,   0])>

In [22]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # attention_hidden_layer shape == (batch_size, 64, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 64, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
#         self.vgg = VGG16(
#                             include_top=False,
#                             weights="imagenet",
#                             input_tensor=layers.Input(embedding_dim),
#                         )
        self.flat = layers.Flatten()
        self.fc = tf.keras.layers.Dense(1000)
        self.fc1 = tf.keras.layers.Dense(100)

    def call(self, x):
#         x = self.vgg(x)
        x = self.flat(x)
        x = self.fc(x)
        x = self.fc1(x)
        x = tf.nn.relu(x)
        return x

class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [24]:
embedding_dim = (224,224,3)
units = 1024
vocab_size = vocab_size
num_steps = len(X_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(100, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

loss_plot = []
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])
#   print(target.shape)

  dec_input = tf.expand_dims([tokenizer1.word_index['sos']] * target.shape[0], 1)
#   print(dec_input)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
#           print(target[:, i])
          dec_input = tf.expand_dims(target[:, i], 1)
#           print(dec_input)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

import time
start_epoch = 0
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(train_dataloader):
#         print(batch)
#         print(img_tensor[0].shape)
#         print(target.shape)
        batch_loss, t_loss = train_step(img_tensor[0], target)
        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = batch_loss.numpy()/int(target.shape[1])
            print(f'Epoch {epoch+1} Batch {batch} Loss {average_batch_loss:.4f}')
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    print(f'Epoch {epoch+1} Loss {total_loss/num_steps:.6f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

Epoch 1 Batch 0 Loss 4.0003
Epoch 1 Batch 100 Loss 1.0423
Epoch 1 Batch 200 Loss 1.7116
Epoch 1 Batch 300 Loss 2.2042
Epoch 1 Batch 400 Loss 2.6954
Epoch 1 Batch 500 Loss 2.3899
Epoch 1 Batch 600 Loss 2.3675
Epoch 1 Batch 700 Loss 2.0291
Epoch 1 Batch 800 Loss 1.7893
Epoch 1 Batch 900 Loss 0.9339
Epoch 1 Batch 1000 Loss 3.8961


KeyboardInterrupt: 

In [ ]:
def evaluate(image, max_length):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0],
                                                 -1,
                                                 img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['sos']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input,
                                                         features,
                                                         hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == 'endseq':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
img = cv2.imread("test4.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)
img = img/255.0
Argmax_Search = evaluate(img,20)